In [1]:
%matplotlib widget
from zMag_Field import *
import numpy as np
import magpylib as magpy


<IPython.core.display.Javascript object>

In [ ]:
def rotate_vec(vec, angle_rad):
    """Rotate a 3D vector (x, y, z) around Z-axis by angle_rad."""
    x, y, z = vec
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    return np.array([
        cos_a * x - sin_a * y,
        sin_a * x + cos_a * y,
        z
    ])

def compute_magnetic_field_antisym(coil_pillar, position):
    """
    Compute magnetic field at 'position' using one base pillar and your specified symmetry.

    Parameters:
        coil_pillar (list): Coils in one base pillar (with +current).
        position (tuple): (x, y, z) field evaluation point.

    Returns:
        np.array: Magnetic field vector [Bx, By, Bz] in Tesla.
    """
    total_B = np.array([0.0, 0.0, 0.0])

    # Your desired symmetry: 1 & 2 (+), 3 & 4 (−)
    symmetry_transforms = [
        (0, +1),            # (-x, -y)
        (np.pi/2, +1),      # (-x, +y)
        (np.pi, -1),        # (+x, +y)
        (3*np.pi/2, -1)     # (+x, -y)
    ]


    for angle, sign in symmetry_transforms:
        rotated_pos = rotate_vec(position, -angle)

        B_local = np.array([0.0, 0.0, 0.0])
        for coil in coil_pillar:
            B_local += coil.getB(rotated_pos)

        B_global = rotate_vec(B_local, angle)
        total_B += sign * B_global
        
        print(total_B)
    return total_B


In [3]:
# === Generate ONE Base Pillar ===
coil_pillar = []

base_position = (0, 0, 0)  # bottom-left
diameter_values = np.linspace(0.007, 0.019, 34)[::-1]

for diameter in diameter_values:
    coil = make_coil(
        current = 5,  # positive current only
        num_coils=30,
        coil_spacing=0.00033,
        position=base_position,
        diameter=diameter
    )
    coil_pillar.extend(coil)  # <- use extend to flatten


In [4]:
# Compute magnetic field at some point
field_point = (0.0, 0.0, 0.03)  # e.g., center 1 cm above

B = compute_magnetic_field_antisym(coil_pillar, field_point)
print("Magnetic field at", field_point, "is", B, "Tesla")


Magnetic field at (0.0, 0.0, 0.03) is [0. 0. 0.] Tesla


In [5]:
def compute_magnetic_gradient_xyz(coil_pillar, position, delta=0.001):
    x, y, z = position

    Bx_plus  = compute_magnetic_field_antisym(coil_pillar, (x + delta, y, z))[0]
    Bx_minus = compute_magnetic_field_antisym(coil_pillar, (x - delta, y, z))[0]
    
    By_plus  = compute_magnetic_field_antisym(coil_pillar, (x, y + delta, z))[1]
    By_minus = compute_magnetic_field_antisym(coil_pillar, (x, y - delta, z))[1]
    
    Bz_plus  = compute_magnetic_field_antisym(coil_pillar, (x, y, z + delta))[2]
    Bz_minus = compute_magnetic_field_antisym(coil_pillar, (x, y, z - delta))[2]

    # Now compute the gradients directly
    dBx_dx = (Bx_plus - Bx_minus) / (2 * delta)
    dBy_dy = (By_plus - By_minus) / (2 * delta)
    dBz_dz = (Bz_plus - Bz_minus) / (2 * delta)
    print("Magnetic gradient at", position, "is", dBx_dx, dBy_dy, dBz_dz, "Tesla/m")

    return dBx_dx, dBy_dy, dBz_dz


In [6]:
dBx_dx, dBy_dy, dBz_dz = compute_magnetic_gradient_xyz(coil_pillar, field_point)
magnetic_moment = 2.061670178918302 



Magnetic gradient at (0.0, 0.0, 0.03) is 0.0 0.0 0.0 Tesla/m


In [7]:
print("Magnetic gradient at", field_point, "is", dBx_dx, dBy_dy, dBz_dz, "Tesla/m")

Magnetic gradient at (0.0, 0.0, 0.03) is 0.0 0.0 0.0 Tesla/m


In [8]:
magnetic_moment = compute_magnetic_moment()
print("Magnetic moment is", magnetic_moment, "A m^2")
total_magnetic_force = np.round(dBz_dz * magnetic_moment, 3)


Magnetic moment is 2.061670178918302 A m^2


In [9]:
print("Magnetic force at", field_point, "is", total_magnetic_force, "N")

Magnetic force at (0.0, 0.0, 0.03) is 0.0 N
